In [1]:
from pathlib import Path
from dotenv import load_dotenv
import os

load_dotenv()

import sys
import os
notebook_dir = os.getcwd()
sys.path.append(os.path.abspath(os.path.join(notebook_dir, '..')))

import torch

from training.experiment_config import config
from training.dataloader import get_data_loader

from tqdm import tqdm
import numpy as np
import pandas as pd

In [2]:
train = pd.read_csv(config.CSV_DIR_TRAIN)
train_ones = train[train["label"] == 1].reset_index(drop=True)
train_zeros = train[train["label"] == 0].reset_index(drop=True)

image_loader = get_data_loader(
		data_dir=config.DATADIR,
		dataset=train_ones,
		mode=config.MODE,
		workers=8,
		batch_size=config.BATCH_SIZE,
		size_px=config.SIZE_PX,
		size_mm=config.SIZE_MM,
		rotations=config.ROTATION,
		translations=config.TRANSLATION,
		)

In [3]:
train_ones.head()

,PatientID,SeriesInstanceUID,StudyDate,CoordX,CoordY,CoordZ,LesionID,AnnotationID,NoduleID,Age_at_StudyDate,Gender,label
0,104839,1.2.840.113654.2.55.30936757672243920224031101...,19990102,122.39,71.28,-220.84,1,104839_1_19990102,104839_1,55,Male,1
1,109345,1.2.840.113654.2.55.30950357318430274435755858...,19990102,57.68,-104.72,-62.51,1,109345_1_19990102,109345_1,62,Male,1
2,202952,1.3.6.1.4.1.14519.5.2.1.7009.9004.209354409551...,19990102,87.32,34.90,-80.11,1,202952_1_19990102,202952_1,56,Female,1
3,112901,1.2.840.113654.2.55.20259099074923358727086752...,20000102,-71.53,38.67,-37.77,1,112901_1_20000102,112901_1,69,Female,1
4,211570,1.3.6.1.4.1.14519.5.2.1.7009.9004.195185449252...,20000102,-56.20,-17.22,-71.04,1,211570_1_20000102,211570_1,64,Male,1


In [8]:
for data in image_loader:
	data_id = data['ID'][0]
	print("1")

	image_location = volumes_dir / "luna25_nodule_blocks" / "image" / f"{data_id}.npy"
	metadata_location = volumes_dir / "luna25_nodule_blocks" / "metadata" / f"{data_id}.npy"
	print("2")
	volume = np.load(str(image_location))
	metadata = np.load(str(metadata_location))
	break

KeyboardInterrupt: 

In [ ]:
volume.shape

In [ ]:
metadata

In [ ]:
i = 0
for data in tqdm(image_loader):

	label = data["label"].float().to(device).item()
	inputs = data["image"]

	# guardar nuevo bloque de imagen
	new_AnnotationID = f"1_1_{str(i)}"
	new_image_location = volumes_dir / "luna25_nodule_blocks" / "image" / f"{new_AnnotationID}.npy"
	
	i+=1

  0%|          | 0/448 [00:10<?, ?it/s]


KeyboardInterrupt: 

In [5]:
image

NameError: name 'image' is not defined